# Topics API: Model Classifier
This colab makes it easy to load the TensorFlow Lite model used by Chrome to infer topics from hostnames.

The classifier is stored `model.tflite`. The file `override_list.pd` contains the topics of 50k hostnames manually annotated by Google.



# Libraries

In [2]:
%pip install grpcio

Defaulting to user installation because normal site-packages is not writeable
  Using cached grpcio-1.62.2.tar.gz (26.3 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [852 lines of output]
      Found cython-generated files...
      running bdist_wheel
      running build
      running build_py
      running build_project_metadata
      creating pyb
      creating pyb/lib.macosx-10.15-x86_64-cpython-310
      creating pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_typing.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_compression.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_channel.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/python/grpcio/grpc/_common.py -> pyb/lib.macosx-10.15-x86_64-cpython-310/grpc
      copying src/pytho

In [3]:
%pip install tflite-support-nightly protobuf-compiler


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 MB 2.6 MB/s eta 0:00:0000:0100:01
  Using cached protobuf_compiler-1.0.20-py3-none-any.whl (8.6 kB)
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Using cached sounddevice-0.4.6-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached tqdm-4.31.1-py2.py3-none-any.whl (48 kB)
  Using cached grpcio-tools-1.18.0.tar.gz (1.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached bleach-2.1-py2.py3-none-any.whl (27 kB)
  Using cached grpcio-

In [9]:
proto_path = 'page_topics_override_list.proto'

example_proto = b"""
syntax = "proto2";

// The whole override list.
message PageTopicsOverrideList {
  repeated PageTopicsOverrideEntry entries = 1;
}

// An individual entry in the override list. |domain| is expected to be the
// exact string input that is otherwise passed to the TFLite model, with all
// needed pre-processing and/or cleaning done to it already.
message PageTopicsOverrideEntry {
  optional string domain = 1;
  optional AnnotatedPageTopics topics = 2;
}

// The topic identifiers for the given domain.
message AnnotatedPageTopics {
  repeated int32 topic_ids = 1;
}
"""

with open(proto_path, 'wb') as f:
  f.write(example_proto)

!cd /tmp/
!protoc "page_topics_override_list.proto" --python_out=.
%cd /Users/abirbenzaamia/Downloads/topics-api-simulator/topics-classification/classifier/
!rm "override_list.pb"
!gzip -dk "override_list.pb.gz"

/Users/abirbenzaamia/Downloads/topics-api-simulator/topics-classification/classifier
rm: override_list.pb: No such file or directory


/Users/abirbenzaamia/Library/Python/3.10/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Override list

In [6]:
import page_topics_override_list_pb2

# Read the existing address book.
override_list = page_topics_override_list_pb2.PageTopicsOverrideList()
try:
  f = open("override_list.pb", "rb")
  override_list.ParseFromString(f.read())
  f.close()
except IOError:
  print("Could not open file.")

# Replaces a set of common domain characters with white space. See https://source.chromium.org/chromium/chromium/src/+/main:components/optimization_guide/core/page_topics_model_executor.cc;l=211?q=meaningless%20f:optimization_guide&ss=chromium
def process_domain(domain):
  replace_chars = ['-', '_', '.', '+']
  for rc in replace_chars:
    domain = domain.replace(rc, " ")
  return domain

def check_override_list(override_list, domain):
  if override_list is None:
    return None
  for entry in override_list.entries:
    if entry.domain == domain:
      return entry.topics.topic_ids
  return None


# Taxonomy

In [8]:
import json

def read_json_to_map(filename):
    map_list={}
    with open(filename, 'r') as file:
        data_map = json.load(file)
    return {int(key): value for key, value in data_map.items()} 

# Example usage:
cat_map = read_json_to_map('../data/taxonomy_v2.json')
#print(cat_map)

# Model Execution


In [42]:
%pip install tflite_support

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 MB 20.3 MB/s eta 0:00:0000:0100:01
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached sounddevice-0.4.6-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl (107 kB)
  Using cached cffi-1.16.0-cp310-cp310-macosx_10_9_x86_64.whl (182 kB)

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tflite_support.task
from tflite_support.task import text
from tflite_support.task import core

# Remember, this will fail if you haven't uploaded model.tflite. 
# See instructions at the top of this file.

options = text.BertNLClassifierOptions(
    base_options=core.BaseOptions(
        file_name='model.tflite'))

tflite_topics = text.BertNLClassifier.create_from_options(options)

def CategorySort(elem):
  return elem.score

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Filtering returned topcis

In [48]:
#Filtering strategy of Chrome 
def filter_top_topics(topics):
    """
    Chrome filtering algorithm to get the approproate topics as in `chrome://topics-internals`
    """
    max_topics = 5
    min_unknown_score = 0.8
    min_topic_score = 0.01
    min_normalized_score = 0.25
    filtered_topics = []

    # Intialization 
    scores_sum = 0
    unknown_topic_score = 0
    for t in topics:
        scores_sum += t.score
        if t.category_name == -2:
            unknown_topic_score = t.score

    # If the Unknown topic score is high 
    if (unknown_topic_score / scores_sum) > min_unknown_score:
        return [-2]
    
    for t in topics: 
        if (t.category_name != '-2') & (t.score > min_topic_score) & (t.score/scores_sum >= min_normalized_score):
            filtered_topics.append(int(t.category_name))
    

    if not filtered_topics:
        return [-2]    #print(t.category_name)
    else:
        return filtered_topics
    


In [67]:
# for domain in domains:
#   print("domain: ", domain)
#   processed_domain = process_domain(domain)
#   print(processed_domain)
#   #check if the domain already exists in the override list manually curated
#   topics = check_override_list(override_list, processed_domain)
#   if topics != None:
#     print(topics)
#     for c in topics:
#       print("Category: {} \t".format( cat_map[c]))
#   #classify the domain using the Google Classifier
#   else:
#     topics = tflite_topics.classify(processed_domain)
#     cats = sorted(topics.classifications[0].categories, key=CategorySort)[-5:][::-1]
#     #print(cats)
#     # for c in cats:
#     #   print("Category: {} \t - Score: {}".format( cat_map[int(c.category_name)],  c.score))

#     filtered_cats = filter_top_topics(cats)
#     for c in filtered_cats:
#       print("Category: {}".format(cat_map[int(c)]))
      
#   print("\n")

In [51]:
def get_domain_topics(domain):
  """Return inferred topics from website hostname using the Topics API classifier """
  print("domain: ", domain)
  processed_domain = process_domain(domain)
  #print(processed_domain)
  #check if the domain already exists in the override list manually curated
  topics = check_override_list(override_list, processed_domain)
  if topics != None:
    #print(topics)
    if not topics:
      return[-2]
    return topics
  #classify the domain using the Google Classifier
  else:
    topics = tflite_topics.classify(processed_domain)
    cats = sorted(topics.classifications[0].categories, key=CategorySort)[-5:][::-1]
    #print(cats)
    # for c in cats:
    #   print("Category: {} \t - Score: {}".format( cat_map[int(c.category_name)],  c.score))
    filtered_cats = filter_top_topics(cats)
    #print(type(filtered_cats))
    #print(filtered_cats)
    #for c in filtered_cats:
      #print("Category: {}".format(cat_map[int(c)]))
    
    return filtered_cats

# Classify the Tranco top-list

In [64]:
import pandas as pd

# read the dataset
df = pd.read_csv("../data/top-1m.csv", header=None)
#inferred_topics_df = pd.DataFrame(columns=['domain','topics'])
lst = []
for ind in range(20000):
    domain = df[1][ind]
    topics_ids = get_domain_topics(domain)
    lst.append([None, None])
    lst[ind][0] = domain
    lst[ind][1] = topics_ids

inferred_topics_df = pd.DataFrame(lst)   
inferred_topics_df.columns = ['domain','topics']
inferred_topics_df.to_csv('../data/top-20k-topics')

domain:  google.com
domain:  facebook.com
domain:  a-msedge.net
domain:  amazonaws.com
domain:  microsoft.com
domain:  apple.com
domain:  googleapis.com
domain:  youtube.com
domain:  akamaiedge.net
domain:  akamai.net
domain:  azure.com
domain:  twitter.com
domain:  instagram.com
domain:  gstatic.com
domain:  cloudflare.com
domain:  googlevideo.com
domain:  live.com
domain:  office.com
domain:  linkedin.com
domain:  tiktokcdn.com
domain:  doubleclick.net
domain:  akadns.net
domain:  googletagmanager.com
domain:  apple-dns.net
domain:  fbcdn.net
domain:  amazon.com
domain:  windowsupdate.com
domain:  microsoftonline.com
domain:  l-msedge.net
domain:  root-servers.net
domain:  icloud.com
domain:  googleusercontent.com
domain:  wikipedia.org
domain:  trafficmanager.net
domain:  fastly.net
domain:  bing.com
domain:  netflix.com
domain:  gtld-servers.net
domain:  wordpress.org
domain:  domaincontrol.com
domain:  digicert.com
domain:  youtu.be
domain:  mail.ru
domain:  github.com
domain:  ui